## 训练深度神经网络有何难点？

- 梯度消失/爆炸问题：在反向传播过程中，梯度变得越来越小或越来越大。二者都会使较浅层难以训练；

- 要训练一个庞大的神经网络，但是数据量不足，或者标注成本很高；

- 训练可能非常慢；

- 具有数百万参数的模型将会有严重的过拟合训练集的风险，特别是在训练实例不多或存在噪音时。

## 梯度消失/爆炸问题

反向传播算法的工作原理是从输出层到输入层，传播误差的梯度。 一旦该算法已经计算了网络中每个参数的损失函数的梯度，它就通过梯度下降使用这些梯度来更新每个参数。

随着算法进展到较低层，梯度往往变得越来越小。 结果，梯度下降更新使得低层连接权重实际上保持不变，并且训练永远不会收敛到最优解。 这被称为梯度消失问题。 在某些情况下，可能会发生相反的情况：梯度可能变得越来越大，许多层得到了非常大的权重更新，算法发散，这是梯度爆炸的问题。

虽然很早就观察到这种现象了（这是造成深度神经网络在2000年早期被抛弃的原因之一），但直到 2010 年左右，人们才才略微清楚了导致梯度消失/爆炸的原因。 Xavier Glorot 和 Yoshua Bengio 发表的题为《Understanding the Difficulty of Training Deep Feedforward Neural Networks》（http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf）的论文发现了一些疑点，包括流行的 sigmoid 激活函数和当时最受欢迎的权重初始化方法的组合，即随机初始化时使用平均值为 0，标准差为 1 的正态分布。简而言之，他们表明，用这个激活函数和这个初始化方案，每层输出的方差远大于其输入的方差。随着网络前向传播，每层的方差持续增加，直到激活函数在顶层饱和。logistic函数的平均值为 0.5 而不是 0（双曲正切函数的平均值为 0，表现略好于深层网络中的logistic函数），使得情况更坏。

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g990h2ym7oj30xc0lhgos.jpg)

看一下logistic激活函数，可以看到当输入变大（负或正）时，函数饱和在 0 或 1，导数非常接近 0。因此，当反向传播开始时， 它几乎没有梯度通过网络传播回来，而且由于反向传播通过顶层向下传递，所以存在的小梯度不断地被稀释，因此较低层得到的改善很小。

### 解决方法

Xavier初始化或Glorot初始化

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g991q8lzerj30ji032mxf.jpg)

LeCun初始化

fan<sub>avg</sub>替换为fan<sub>in</sub>，就得到了Yann LeCun在1990年代提出的初始化策略，他称其为LeCun初始化。

He 初始化

ReLU 激活函数（及其变体，包括简称 ELU 激活）的初始化策略有时称为 He 初始化

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g991sa8iumj30qk09s0u6.jpg)

In [2]:
# 默认情况下，Keras使用均匀分布的Glorot初始化函数。创建层时，
# 可以通过设置kernel_initializer="he_uniform"或kernel_initializer="he_normal"变更为He初始化

from tensorflow import keras

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

## 非饱和激活函数

Glorot 和 Bengio 在 2010 年的论文中的一个见解是，消失/爆炸的梯度问题部分是由于激活函数的选择不好造成的。 在那之前，大多数人都认为，如果大自然选择在生物神经元中使用 sigmoid 激活函数，它们必定是一个很好的选择。 但事实证明，其他激活函数在深度神经网络中表现得更好，特别是 ReLU 激活函数，主要是因为它对正值不会饱和（也因为它的计算速度很快）。

但是，ReLU激活功能并不完美。 它有一个被称为 “ReLU 死区” 的问题：在训练过程中，一些神经元会“死亡”，即它们停止输出 0 以外的任何东西。在某些情况下，你可能会发现你网络的一半神经元已经死亡，特别是使用大学习率时。 在训练期间，如果神经元的权重得到更新，使得神经元输入的加权和为负，则它将开始输出 0 。当这种情况发生时，由于当输入为负时，ReLU函数的梯度为0，神经元就只能输出0了。

### leaky ReLU

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g9923btiyuj30xc0lgwh4.jpg)

```
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    [...]
])
```

### ELU

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g99252a9vhj30xc0la76e.jpg)

```
keras.layers.ELU(alpha=1.0)
```

## 批归一化

```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
```

## 复用预训练层

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g99835gz4ej30xc0psq6v.jpg)

## 无监督预训练

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g99846g4m6j30xc0m1n0x.jpg)

## 更快的优化器

### 动量优化(1964)

梯度用作加速度，不用作速度。 

```
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)
```

### Nesterov 加速梯度(1983)

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g998f1m5f3j30xc0izwgr.jpg)

可以看到，Nesterov 更新稍微靠近最佳值。 过了一段时间，这些小的改进加起来，NAG 最终比常规的动量优化快得多。 此外，当动量推动权重横跨山谷时，▽1继续推进越过山谷，而▽2推回山谷的底部。 这有助于减少振荡，从而更快地收敛。

```
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```

### AdaGrad

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g998gay6q7j30xc0h40uk.jpg)

## 学习率调整

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g998jih8shj30xc0g740v.jpg)

幂调度

```
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
```

指数调度

```
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)
```

预定的分段恒定学习率

```
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001
```

性能调度

```
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
```


```
# tf.keras专有的
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)
```

## 通过正则化避免过拟合

### dropout 早停

dropout是深度神经网络最流行的正则化方法之一。 它由 Geoffrey Hinton 于 2012 年提出，并在 Nitish Srivastava 等人的2014年论文中进一步详细描述，并且已被证明是非常成功的：即使是最先进的神经网络，仅仅通过增加dropout就可以提高1-2％的准确度。

这是一个相当简单的算法：在每个训练步骤中，每个神经元（包括输入神经元，但不包括输出神经元）都有一个暂时“丢弃”的概率p，这意味着在这个训练步骤中它将被完全忽略， 在下一步可能会激活。 超参数p称为丢失率，通常设为 10%到50%之间；循环神经网络之间接近20-30%，在卷积网络中接近40-50%。 训练后，神经元不会再丢失。

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g998th37ylj30xc0nu41q.jpg)